# rewrite loading xyz using ase

In [1]:
import py3Dmol
import numpy as np
import matplotlib.pyplot as plt
# from IPython.display import display
from ipyfilechooser import FileChooser
import re
import ase
from ase.io import read
import plotly.graph_objects as go
import numpy as np

%matplotlib widget

def shrink_cluster(elements, coordinates, radius):
    print(f"Reduce cluster size to less than {radius}")
    print("Old cluster size is")
    distances_all, size = cluster_size_calc(elements, coordinates)
    indices = np.where(distances_all < radius)[0]
    elements_shrinked = [elements[i] for i in indices]
    coordinates_shrinked = [coordinates[i] for i in indices]
    print("New cluster size is")    
    distances_all, size = cluster_size_calc(elements_shrinked, coordinates_shrinked )

    return list(elements_shrinked), coordinates_shrinked


## Examples

In [2]:
def read_file():
    if fc.selected is not None:
        with open(fc.selected, 'r') as file:
            content = file.read()
        return content
    else:
        print("No file selected.")

fc = FileChooser()
display(fc)

FileChooser(path='/Users/juanjuanhuang/Desktop/neighbor', filename='', title='', show_hidden=False, select_des…

In [3]:
# Load the XYZ file
atoms = ase.io.read(fc.value)

# Now you can work with the 'atoms' object
print(atoms)

Atoms(symbols='O4PbO4PbO7PbO4Pb2O8PbO4Pb2O7Pb2O4Pb2O2Pb3', pbc=False)


In [84]:
class ClusterNeighbor(object):
    def __init__(self):
        pass

    def load_xyz(self, xyz_path):
        self.atoms = ase.io.read(fc.value)
        self.elements = [test.atoms[i].symbol for i in range(len(self.atoms))]
        self.elements_num = {element_i: self.elements.count(element_i) for element_i in set(self.elements)}
        self.element_index_group = {element_set_i: [i for i, element_i in enumerate(test.elements) if element_i == element_set_i] for element_set_i in set(self.elements)}
        
    def view_xyz(self, style_all=None, highlight_atom1="O", highlight_atom2="Pb", label=True):
        self.xyz_string = f"{len(self.atoms)}\n\n" 
        for atom in self.atoms:
            self.xyz_string += f"{atom.symbol} {atom.position[0]} {atom.position[1]} {atom.position[2]}\n"
        
        self.view = py3Dmol.view(width=500,height=500)
        self.view.addModel(self.xyz_string,'xyz',)
        if style_all is None:
            style_all = {'stick':{'radius':.1, 'alpha':0.2, 'color':'gray'}, 
                         'sphere': {'radius':.3}
                        }
        self.view.setStyle(style_all)
        
        self.view.addStyle({'atom': highlight_atom1}, 
                           {'sphere': {'color': 'red', 'radius': 0.5}})  
        
        self.view.addStyle({'atom': highlight_atom2}, 
                           {'sphere': {'color': 'blue', 'radius': 0.3}})  
        self.view.setBackgroundColor('0xeeeeee')
        if label:
            for i, atom_i in enumerate(self.atoms):
                self.view.addLabel(f"{i}", {'position': {'x': atom_i.position[0], 'y': atom_i.position[1], 'z': atom_i.position[2]}, 
                                    'fontColor': 'k', 'fontSize': 12, 'backgroundColor': 'white', 'backgroundOpacity':0.5})
        self.view.zoomTo()
        self.view.show()
        self.view.title(self.atoms.get_chemical_formula())
    
    def get_cluster_size(self):
        self.cluster_size = self.atoms.get_all_distances().max()/2
        print(f"Cluster size is {self.cluster_size} A")
        return self.cluster_size
    
    def get_pairs(self):
        self.pairs_index = [(i, j) for i in range(len(self.atoms)) for j in range(i + 1, len(self.atoms))]
        # self.pairs_index = [(i, j) for i in range(len(self.atoms)) for j in range(len(self.atoms))]
        self.pairs_element = [sorted([self.atoms[i].symbol, self.atoms[j].symbol]) for i, j in self.pairs_index]
        self.pairs = [f"{atom_i}-{atom_j}" for atom_i, atom_j in self.pairs_element]
        self.pairs_unique = [f"{self.atoms[i].symbol}({self.atoms[i].index})-{self.atoms[j].symbol}({self.atoms[j].index})" for i, j in self.pairs_index]
        self.distance_all = [self.atoms.get_all_distances()[i][j] for i, j in self.pairs_index]
        self.pairs_types = set(self.pairs)
        
        self.pairs_group = {key: {'pairs_index':[],
                                  'pairs':[],
                                  'pairs_unique':[],
                                  'distance':[]} for key in self.pairs_types}
        
        for i in range(len(self.pairs)):
            self.pairs_group[self.pairs[i]]['pairs_index'].append(self.pairs_index[i])
            self.pairs_group[self.pairs[i]]['pairs_unique'].append(self.pairs_unique[i])
            self.pairs_group[self.pairs[i]]['distance'].append(self.distance_all[i])
        return self.pairs_group

    def get_CN(self, center_atom=None, error_bar=0.01):
        if not hasattr(self, 'pairs_types'):
            self.get_pairs()
            
        if not hasattr(self, 'CN_distances'):
            self.CN_distances = {}
            self.CN = {}
                    
        if center_atom is None:
            center_atom = self.atoms[0].symbol
                
        for pair_i in self.pairs_types:
            if center_atom in pair_i:
                distance_sorted = np.array(sorted(self.pairs_group[pair_i]['distance']))
                diff = np.diff(distance_sorted)
                indices = np.where(diff > error_bar)[0] + 1
                self.CN_distances[pair_i] = np.split(distance_sorted, indices)
                self.CN[pair_i] = {np.average(group): group.shape[0]*2/self.elements_num[center_atom] for group in self.CN_distances[pair_i]}
    
    def get_CN_new(self, center_atom=None, CN_atom=None, error_bar=0.01):
        if not hasattr(self, 'pairs_types'):
            self.get_pairs()
            
        if not hasattr(self, 'CN_distances'):
            self.CN_distances = {}
            self.CN = {}
                    
        if center_atom is None:
            center_atom = self.atoms[0].symbol
        
        if CN_atom is None:
            center_atom = self.atoms[1].symbol
                
        for pair_i in self.pairs_types:
            if center_atom in pair_i:
                distance_sorted = np.array(sorted(self.pairs_group[pair_i]['distance']))
                diff = np.diff(distance_sorted)
                indices = np.where(diff > error_bar)[0] + 1
                self.CN_distances[pair_i] = np.split(distance_sorted, indices)
                self.CN[pair_i] = {np.average(group): group.shape[0]*2/self.elements_num[center_atom] for group in self.CN_distances[pair_i]}
    
    
    # def get_all_distances()
    def plot_hist(self, binsize=0.2):
        if not hasattr(self, 'pairs_types'):
            self.get_pairs()      
        fig = go.Figure()
        for key_i in self.pairs_group.keys():
            fig.add_trace(go.Histogram(x=self.pairs_group[key_i]['distance'], name=key_i, opacity=0.6, 
                                       xbins={'size':binsize},marker={'line':{'color':'white','width':2}}))

        fig.update_layout(
            xaxis_title_text='Distances [A]', yaxis_title_text='pairs',
            plot_bgcolor='rgba(0.02,0.02,0.02,0.02)',  # Transparent plot background
            xaxis={'tickmode':'auto'}, barmode='overlay',  # Overlay histograms,
            width=600, height=400)
        fig.show()

In [71]:
test.elements_num

{'O': 44, 'Pb': 15}

In [79]:
indices = [i for i, element_i in enumerate(test.elements) if element_i == 'Pb']


In [ ]:
indices

In [81]:
print(indices)

[4, 9, 17, 22, 23, 32, 37, 38, 46, 47, 52, 53, 56, 57, 58]


In [73]:
indices

[0,
 1,
 2,
 3,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 48,
 49,
 50,
 51,
 54,
 55]

[0,
 1,
 2,
 3,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 48,
 49,
 50,
 51,
 54,
 55]

In [58]:
test.atoms.get_distances(1,[2,3,4])

array([3.05665632, 3.05665572, 2.16928433])

In [91]:
print(test.element_index_group['O'])
print(test.element_index_group['Pb'])

[0, 1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 54, 55]
[4, 9, 17, 22, 23, 32, 37, 38, 46, 47, 52, 53, 56, 57, 58]


In [85]:
test = ClusterNeighbor()
test.load_xyz(fc.value)
test.view_xyz(highlight_atom1="Pb", highlight_atom2="O", label=True)
test.get_cluster_size()
test.atoms.get_angle(1,4,0)
test.get_pairs()
test.get_CN(center_atom='O', error_bar=0.01)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Cluster size is 5.926372577426853 A


In [46]:
test.atoms.get_distance(9, 8)

2.153451725543435

In [48]:
test.CN['O-Pb']

{2.15345127233584: 2.3636363636363638,
 2.1692843565604814: 4.7272727272727275,
 3.759464103600722: 3.272727272727273,
 4.016148774568188: 3.272727272727273,
 4.450974934970655: 5.818181818181818,
 4.861048023898766: 1.2727272727272727,
 5.062180336008796: 5.090909090909091,
 5.262128217289388: 2.1818181818181817,
 5.909023530218671: 2.1818181818181817,
 5.926372243004845: 2.1818181818181817,
 6.229254614092598: 3.6363636363636362,
 6.541817358979242: 2.909090909090909,
 6.659154270243802: 1.8181818181818181,
 7.113771779844665: 0.7272727272727273,
 7.340720537707306: 3.272727272727273,
 7.472377961033679: 2.1818181818181817,
 7.609254687726023: 0.7272727272727273,
 7.752545874266088: 1.4545454545454546,
 7.860522288892568: 2.1818181818181817,
 8.08286153850564: 1.4545454545454546,
 8.342552820814575: 0.7272727272727273,
 8.536897202414814: 1.0909090909090908,
 8.582446934276874: 0.36363636363636365,
 8.768876438013006: 0.7272727272727273,
 9.073544419622824: 0.36363636363636365,
 9.17

In [10]:
test.get_CN(center_atom='Pb', error_bar=0.01)
test.CN['O-Pb']
test.get_CN(center_atom='O', error_bar=0.01)
test.CN['O-O']
test.get_CN(center_atom='Pb', error_bar=0.01)
test.CN['Pb-Pb']

test.plot_hist(binsize=0.1)


In [43]:
test.CN['O-O']


{2.707596806026806: 0.7272727272727273,
 3.056656193800888: 5.090909090909091,
 3.389999705849734: 1.2727272727272727,
 3.5972552235297384: 0.9090909090909091,
 4.306902524919862: 0.5,
 4.338568791573196: 1.0,
 4.942908473302007: 1.4545454545454546,
 4.95999997915164: 1.8181818181818181,
 5.323117733597045: 2.909090909090909,
 5.4810132827879965: 0.7272727272727273,
 5.685714003439255: 2.909090909090909,
 6.007803167949652: 2.5454545454545454,
 6.289190064263103: 2.1818181818181817,
 6.779999746861404: 0.5454545454545454,
 7.014499239886487: 0.9090909090909091,
 7.1361835543201435: 0.5454545454545454,
 7.163782564669132: 1.8181818181818181,
 7.300649072310556: 0.5454545454545454,
 7.518928314191071: 0.2727272727272727,
 7.6515585975731595: 1.4545454545454546,
 7.6751961374894755: 0.7272727272727273,
 7.790718763566937: 1.2727272727272727,
 7.905044932820843: 1.8181818181818181,
 8.032297339241033: 0.2727272727272727,
 8.231197392663079: 0.18181818181818182,
 8.247810850348154: 0.363636

In [18]:
atoms.pop?

Signature: atoms.pop(i=-1)
Docstring: Remove and return atom at index *i* (default last).
File:      ~/.local/lib/python3.10/site-packages/ase/atoms.py
Type:      method

In [36]:
test2 = ClusterNeighbor()
test2.load_xyz(fc.value)
for i in range(47):
    test2.atoms.pop()

test2.view_xyz(highlight_atom1="Pb", highlight_atom2="O", label=True)
test2.get_cluster_size()
test2.get_pairs()
test2.get_CN(center_atom='O', error_bar=0.02)
test2.CN['O-Pb']

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Cluster size is 3.6503248720208448 A


{2.1640067315448177: 0.5454545454545454,
 4.016148350175418: 0.18181818181818182,
 5.262128024409925: 0.18181818181818182}

In [38]:
test2.atoms.get_distance(3,4)

2.153451309638553

In [42]:
ase.io.write('reduced.xyz', test2.atoms)

In [ ]:
test2.atoms.get_distance(51,49)


In [ ]:
0.4 * test.elements_num['Pb']

In [ ]:

elements = [xyz_sorted[i][0] for i in range(len(compound))]

coordinates = [np.array([float(xyz_sorted[i][1]),
                         float(xyz_sorted[i][2]), 
                         float(xyz_sorted[i][3])]) for i in range(len(compound))]


num_atoms = len(elements)
output = {"unique_pairs_all":{},
          "pairs_all":[],
          "unique_element":list(set(elements)),
          "element_num":{},
          "bond_num":{}
          }

for i in range(num_atoms):
    atom_i = elements[i]
    atom_unique_i = f"{elements[i]}({i:d})"
    for j in range(num_atoms):
        if j !=i:
            atom_j = elements[j]
            atom_unique_j = f"{elements[j]}({j:d})"
            distance = calculate_distance(coordinates[i], coordinates[j])
            output['pairs_all'].append(f"{atom_i}-{atom_j}")
            output['unique_pairs_all'][f"{atom_unique_i}-{atom_unique_j}"] = distance
            # print(f"{atom_unique_i}-{atom_unique_j}: {distance} Å")

output["bond_types"] = list(set(output['pairs_all']))

for element_i in output["unique_element"]:
    output['element_num'][element_i] = elements.count(element_i)


bond_type = output["bond_types"]

for bond_i in output["bond_types"]:
    # print(bond_i)
    element1, element2 = bond_i.split("-")
    pattern = rf'{element1}\(\d+\)-{element2}\(\d+\)'
    output[f"{bond_i}_num"] = {}
    for key_i in output['unique_pairs_all'].keys():
        if re.match(pattern, key_i):
            output[f"{bond_i}_num"][key_i] = output['unique_pairs_all'][key_i]


In [ ]:
plt.figure(figsize=(10,3))
for key_i in output['bond_types']:
    plt.hist(output[f"{key_i}_num"].values(), bins=80, alpha=0.3, edgecolor='white', label=key_i)
plt.xlabel("Distance [Å]")
plt.ylabel("Number of pairs")
plt.legend()
plt.ylim(0, 180)
plt.tight_layout()
plt.show()

# within 0.001 A is considered as same distance, cakculate all distances

In [ ]:
def CN_calc(R_range=[1,3], pair=None):
    shell_distances = {}
    element = pair.split("-")[0]
    for key_i in output[f"{pair}_num"].keys():
        if R_range[0] < output[f"{pair}_num"][key_i] < R_range[1]:
            shell_distances[key_i] = output[f"{pair}_num"][key_i]
    CN_num = len(shell_distances.values())/output['element_num'][element]
    print(f"Average CN of {element} element: {CN_num}")
    average_distance = np.average(np.array(list(shell_distances.values())))
    print(f"Average distance of {pair} pair: {average_distance} \n")
    return shell_distances, average_distance


result = CN_calc(R_range=[1,5], pair="O-O")
# result = CN_calc(R_range=[3,3.1], pair="O-O")
# result = CN_calc(R_range=[3.3,3.1], pair="O-O")

# result = CN_calc(R_range=[2,2.16], pair="O-Pb")
# result = CN_calc(R_range=[2.16,3.7], pair="O-Pb")

In [ ]:
from operator import itemgetter

data = output["Pb-O_num"]

# Sort the dictionary by its values
sorted_items = sorted(data.items(), key=itemgetter(1))
sorted_data = dict(sorted_items)

print(sorted_dict)

In [ ]:
def group_data(data, error_bar=0.01):

    sorted_items = sorted(data.items(), key=itemgetter(1))
    data_sorted = dict(sorted_items)
    diff = np.diff(data_sorted.values())
    indices = np.where(diff > error_bar)[0] + 1

    # Split the array at these indices
    return np.split(data_sorted, indices)

# Example data values
data_values = np.array(output['O-O_num'])

# Grouping the data
grouped_data = group_data(data_values)

# Print the grouped data
for group in grouped_data:
    print(group)


In [ ]:
import pandas as pd

data_values = output["Pb-O_num"]

df = pd.DataFrame(list(data_values.items()), columns=['Key', 'Value'])
df = df.sort_values(by='Value')

# Function to group keys based on value difference threshold
def group_keys(df, threshold=0.01):
    # Create an empty list to store groups
    groups = []
    current_group = [df.iloc[0]['Key']]

    for i in range(1, len(df)):
        # Check if the current value is within the threshold of the previous value
        if abs(df.iloc[i]['Value'] - df.iloc[i-1]['Value']) <= threshold:
            current_group.append(df.iloc[i]['Key'])
        else:
            # If not, add the current group to groups and start a new group
            groups.append(current_group)
            current_group = [df.iloc[i]['Key']]

    # Add the last group to groups
    groups.append(current_group)

    return groups

# Grouping the keys
grouped_keys = group_keys(df)

# Print the grouped keys
# for group in grouped_keys:
    # print(group)


In [ ]:
elements_shinked, coordinates_shrinked = shrink_cluster(elements, coordinates, radius=5)

In [ ]:
elements = elements_shinked
coordinates = coordinates_shrinked
num_atoms = len(elements)
output = {"unique_pairs_all":{},
          "pairs_all":[],
          "unique_element":list(set(elements)),
          "element_num":{},
          "bond_num":{}
          }

for i in range(num_atoms):
    atom_i = elements[i]
    atom_unique_i = f"{elements[i]}({i:d})"
    for j in range(num_atoms):
        if j !=i:
            atom_j = elements[j]
            atom_unique_j = f"{elements[j]}({j:d})"
            distance = calculate_distance(coordinates[i], coordinates[j])
            output['pairs_all'].append(f"{atom_i}-{atom_j}")
            output['unique_pairs_all'][f"{atom_unique_i}-{atom_unique_j}"] = distance
            # print(f"{atom_unique_i}-{atom_unique_j}: {distance} Å")

output["bond_types"] = list(set(output['pairs_all']))
for element_i in output["unique_element"]:
    output['element_num'][element_i] = elements.count(element_i)


bond_type = output["bond_types"]

for bond_i in output["bond_types"]:
    # print(bond_i)
    element1, element2 = bond_i.split("-")
    pattern = rf'{element1}\(\d+\)-{element2}\(\d+\)'
    output[f"{bond_i}_num"] = {}
    for key_i in output['unique_pairs_all'].keys():
        if re.match(pattern, key_i):
            output[f"{bond_i}_num"][key_i] = output['unique_pairs_all'][key_i]



In [ ]:
plt.figure(figsize=(10,3))
for key_i in output['bond_types']:
    plt.hist(output[f"{key_i}_num"].values(), bins=80, alpha=0.3, edgecolor='white', label=key_i)
plt.xlabel("Distance [Å]")
plt.ylabel("Number of pairs")
plt.legend()
plt.ylim(0, 180)
plt.tight_layout()
plt.show()

In [ ]:
def CN_calc(R_range=[1,3], pair=None):
    shell_distances = {}
    element = pair.split("-")[0]
    for key_i in output[f"{pair}_num"].keys():
        if R_range[0] < output[f"{pair}_num"][key_i] < R_range[1]:
            shell_distances[key_i] = output[f"{pair}_num"][key_i]
    CN_num = len(shell_distances.values())/output['element_num'][element]
    print(f"Average CN of {element} element: {CN_num}")
    average_distance = np.average(np.array(list(shell_distances.values())))
    print(f"Average distance of {pair} pair: {average_distance} \n")
    return shell_distances, average_distance


result = CN_calc(R_range=[1,3], pair="O-O")
result = CN_calc(R_range=[3,3.1], pair="O-O")
result = CN_calc(R_range=[3.3,3.1], pair="O-O")

result = CN_calc(R_range=[2,2.16], pair="Pb-O")
result = CN_calc(R_range=[2.16,3.7], pair="Pb-O")

75/number of Pd num_atoms (coordinations )

1. CN of Pd atoms, CN of Oxygen
2. calculate radius of the cluster: longest distance from (0,0,0)
3. increase radius of the cluster, doesn't need to shrink/expand the bond lengths, chop the hist and calculate the CN again
4. cif file --> cluster, increase clusters

Display local file.

In [ ]:
view = py3Dmol.view(query='pdb:1ycr')
view.setStyle({'cartoon': {'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}})

In [ ]:
import requests, base64
r = requests.get('https://mmtf.rcsb.org/v1.0/full/5lgo')
view = py3Dmol.view()
view.addModel(base64.b64encode(r.content).decode(),'mmtf')
view.addUnitCell()
view.zoomTo()
